# Entendendo o problema

Desenvolver um corretor ortográfico que receba como input uma palavra digitada incorretamente e retorne uma sugestão da palavra corrigida.

Exemplo: in: lgica; out: lógica

# Lendo o arquivo .txt

Arquivo 'artigos.txt' que contém apenas palavras em português

In [42]:
# lendo o arquivo .txt:
with open("artigos.txt", 'r', encoding='utf8') as f:
    artigos = f.read()  # armazena o conteúdo do arquivo nessa variável
print(artigos[:200])  # exibe os primeiros 200 caracteres da variável




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o


# Tokenização

Tokenizar, em NLP, se refere a dividir a variável inteira ('artigos') em trechos menores (com o método split(), por exemplo)

Processo muito utilizado no pré-processamento de dados textuais

## Descobrindo quantas palavras a variável tem:

### Mostrando o tamanho da variável (número de caracteres, e não de palavras)

In [2]:
print(f'A variável artigos possui {len(artigos)} caracteres.')

A variável artigos possui 2605046 caracteres.


Problemas detectados: existem caracteres de pontuação no meio. Existem palavras com caracteres em caixa alta. 

## Tokenizando:

In [3]:
import nltk  # importando o pacote
# nltk.download('punkt')  # fazendo download de outro dado necessário do nltk (senão dá erro)

### Exemplo teste do nltk.tokenize.word_tokenize()

#### Tokenizando o exemplo

In [4]:
texto_exemplo = "Olá, tudo bem?"  # string exemplo

In [5]:
palavras_separadas = nltk.tokenize.word_tokenize(texto_exemplo)  # lista com os tokens
print(palavras_separadas)

['Olá', ',', 'tudo', 'bem', '?']


Separou a string em palavras e pontuações, retornando uma lista com todas

#### Removendo o que não é palavra 

In [6]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():  # se não houver pontuação/caractere alfaanumérico...
            lista_palavras.append(token)
    return lista_palavras

A função acima retorna apenas as palavras de determinada lista, removendo caracteres de pontuação ou numéricos

In [7]:
separa_palavras(palavras_separadas)

['Olá', 'tudo', 'bem']

### Voltando e gerando a lista de palavras (sem pontuação)

Aplicando a mesma lógica demonstrada no exemplo acima à variável 'artigos'

In [8]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)  # cria a lista de tokens
lista_palavras = separa_palavras(lista_tokens)  # retorna a lista apenas com palavras
len(lista_palavras)

403104

In [9]:
print(f'A variável artigos possui {len(lista_palavras)} palavras')

A variável artigos possui 403104 palavras


Problema: Existem palavras sem normalização (maiúscula/minúscula)

### Normalizando: removendo maiúsculas

In [10]:
# função para deixar todos os caracteres em letras minúsculas
def normalização(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:        
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

In [11]:
# chamando a função 
lista_normalizada = normalização(lista_palavras)
print(lista_normalizada[:5])

['imagem', 'temos', 'a', 'seguinte', 'classe']


Problema: existem palavras repetidas.

### Removendo palavras duplicadas:

In [12]:
lista_unique = set(lista_normalizada)
len(lista_unique)

18465

# Construindo o corretor

## Corretor:

### Função do corretor que implementa os métodos de correção:

In [13]:
# gera várias palavras novas adicionando letras:
def gerar_palavras(palavra):    
    fatias = []
    for c in range(len(palavra) + 1):        
        fatias.append((palavra[:c], palavra[c:]))        
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letras(fatias)  # concatena as duas operações (deletar e inserir)
    palavras_geradas += troca_letras(fatias)  # concatena de novo
    palavras_geradas += inverte_letras(fatias)  # concatena de novo
    return palavras_geradas

### Calculando a probabilidade de cada palavra no corpus

In [14]:
total_palavras = len(lista_normalizada)  # total de palavras no corpus
freq = nltk.FreqDist(lista_normalizada)  # calcula a frequência (número de vezes) que cada palavra aparece no corpus


# calcula a probabilidade que CADA PALAVRA aparece no corpus de amostra
def probabilidade(palavra_gerada): 
    return freq[palavra_gerada]/total_palavras

### Corretor em si:

In [15]:
def corretor(palavra):
    palavras_geradas = gerar_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

## Métodos de correção:

### Método de correção 1: inserir letra

In [16]:
def insere_letras(fatias):  
    novas_palavras = []    
    for l, r in fatias:  # left & right
        for letra in 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç':
            novas_palavras.append(l + letra + r)
    return novas_palavras

### Método de correção 2: remover letra

In [17]:
def deleta_letras(fatias):
    novas_palavras = []    
    for l, r in fatias:  # left & right
        novas_palavras.append(l + r[1:])
    return novas_palavras

### Método de correção 3: trocar letra

Para quando escrever uma letra errada

In [18]:
def troca_letras(fatias):  
    novas_palavras = []    
    for l, r in fatias:  # left & right        
        for letra in 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç':
            novas_palavras.append(l + letra + r[1:])
    return novas_palavras

### Método de correção 4: inverte letra

Para quando inverter as letras

In [19]:
def inverte_letras(fatias):
    novas_palavras = []      
    for l, r in fatias:  # left & right
        if len(r) > 1:
            novas_palavras.append(l + r[1] + r[0] + r[2:])
    return novas_palavras

### Avaliando o corretor

In [20]:
# função que extrai palavras teste do arquivo 'palavras.txt'
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, "r", encoding='utf-8')
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

In [21]:
lista_teste = cria_dados_teste('palavras.txt')  # lista com palavras teste do arquivo

In [22]:
# contruindo função para avaliar a taxa de acerto do corretor
def avaliador(testes):
    num_palavras = len(testes)
    acertos = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertos += 1
    taxa_acerto = acertos/num_palavras*100
    print(f'Taxa de acerto: acertou {taxa_acerto:.2f}% de um total de {num_palavras} palavras')

#### Testando

In [23]:
avaliador(lista_teste)

Taxa de acerto: acertou 76.34% de um total de 186 palavras


# Testando a taxa de palavras desconhecidas

Função avaliador() atualizada 

In [24]:
def avaliador_2(testes, vocab):
    num_palavras = len(testes)
    acertos = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertos += 1
        elif correta not in vocab:            
            desconhecida += 1
    taxa_acerto = acertos/num_palavras*100
    taxa_desconhecida = desconhecida/num_palavras*100
    print(f'Taxa de acerto: acertou {taxa_acerto:.2f}% de um total de {num_palavras} palavras')
    print(f'Taxa de palavras desconhecidas: {taxa_desconhecida:.2f}%')

vocab = set(lista_normalizada)

In [25]:
avaliador_2(lista_teste, vocab)

Taxa de acerto: acertou 76.34% de um total de 186 palavras
Taxa de palavras desconhecidas: 6.99%


# Criando um corretor "turbinado"

## Função gerador_turbinado() 

Antes corrigia apenas um erro por vez. Agora vai corrigir até dois erros

In [26]:
# novo gerador, que recebe uma lista de palavras geradas e, para cada uma delas, gera novas palavras de novo 
def gerador_turbinado(palavras_geradas):  
    novas_palavras = []
    for palavra in palavras_geradas:  # chama o método gerar_palavras() para cada palavra gerada
        novas_palavras += gerar_palavras(palavra)
    return novas_palavras

### Testando:

In [27]:
palavras_g = gerador_turbinado(gerar_palavras('lóiigica'))
'lógica' in palavras_g

True

#### Verificando o tamanho -> quantidade de palavras geradas pelo gerador_turbinado()


In [30]:
print(len(palavras_g))

691744


## Função novo_corretor()

Diminuindo os candidatos à palavra correta, descartando os menos prováveis

In [31]:
def novo_corretor(palavra):
    palavras_geradas = gerar_palavras(palavra)  # lista com palavras geradas pelo método gerar_palavras()
    palavras_turbinado = gerador_turbinado(palavras_geradas)  # lista com palavras geradas pelo método gerar_turbinado()
    todas_palavras = set(palavras_geradas + palavras_turbinado)  # concatena as duas listas acima
    candidatos = [palavra]  # a priori essa var contém apenas a palavra inicial (caso esteja correta, e não errada)
    for palavra in todas_palavras:
        if palavra in vocab:
            candidatos.append(palavra)
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

### Testando

In [32]:
novo_corretor('lóiigica')

'lógica'

## Novo avaliador()

Corrigindo erros do avaliador prévio e aplicando o novo_corretor()

In [36]:
def avaliador_3(testes, vocab):
    num_palavras = len(testes)
    acertos = desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)  # novo corretor
        desconhecida += (correta not in vocab)
        if palavra_corrigida == correta:
            acertos += 1
    taxa_acerto = acertos/num_palavras*100
    taxa_desconhecida = desconhecida/num_palavras*100
    print(f'Taxa de acerto: acertou {taxa_acerto:.2f}% de um total de {num_palavras} palavras')
    print(f'Taxa de palavras desconhecidas: {taxa_desconhecida:.2f}%')

vocab = set(lista_normalizada)
avaliador_3(lista_teste, vocab)

Taxa de acerto: acertou 55.38% de um total de 186 palavras
Taxa de palavras desconhecidas: 6.99%


# Função avaliadora final: avaliador_4()

Após comparação, o novo_corretor() possui, na verdade, pior performance que o corretor()
- Por isso, a versão final do avaliador vai usar o corretor()

In [37]:
def avaliador_4(testes, vocab):
    num_palavras = len(testes)
    acertos = desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)  # usando o corretor original 
        desconhecida += (correta not in vocab)
        if palavra_corrigida == correta:
            acertos += 1
    taxa_acerto = acertos/num_palavras*100
    taxa_desconhecida = desconhecida/num_palavras*100
    print(f'Taxa de acerto: acertou {taxa_acerto:.2f}% de um total de {num_palavras} palavras')
    print(f'Taxa de palavras desconhecidas: {taxa_desconhecida:.2f}%')

vocab = set(lista_normalizada)
avaliador_4(lista_teste, vocab)

Taxa de acerto: acertou 76.34% de um total de 186 palavras
Taxa de palavras desconhecidas: 6.99%


## Comparando resultados entre as duas versões dos corretores

In [40]:
palavra = 'loogica'  # inserir a palavra teste aqui
print(f'novo_corretor(): {novo_corretor(palavra)}')
print(f'corretor(): {corretor(palavra)}')

novo_corretor(): lógica
corretor(): aloogica
